# diatom_scan calculation style

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

Description updated: 2019-07-26

## Introduction

The diatom_scan calculation style evaluates the interaction energy between two atoms at varying distances.  This provides a measure of the isolated pair interaction of two atoms providing insights into the strengths of the attraction/repulsion and the effective range of interatomic spacings.  This scan also gives insight into the computational smoothness of the potential's functional form.

### Version notes

### Additional dependencies

### Disclaimers

- [NIST disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm)
- No 3+ body interactions are explored with this calculation as only two atoms are used.


## Method and Theory

Two atoms are placed in an otherwise empty system.  The total energy of the system is evaluated for different interatomic spacings.


## Demonstration

### 1. Setup

#### 1.1. Library imports

Import libraries needed by the calculation. The external libraries used are:

- [numpy](http://www.numpy.org/)
- [DataModelDict](https://github.com/usnistgov/DataModelDict)
- [atomman](https://github.com/usnistgov/atomman)
- [iprPy](https://github.com/usnistgov/iprPy)
- [bokeh](http://bokeh.pydata.org/)

In [1]:
# Standard library imports
from pathlib import Path
import os
import sys
import uuid
import shutil
import datetime
from collections import OrderedDict
from math import floor
from copy import deepcopy

from IPython.display import display, clear_output, HTML
from ipywidgets import Dropdown, Text, BoundedIntText, BoundedFloatText, Output

# http://www.numpy.org/
import numpy as np

# https://github.com/usnistgov/DataModelDict 
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

# http://bokeh.pydata.org/
import bokeh
print('Bokeh version =', bokeh.__version__)
from bokeh.plotting import figure, output_file, show
from bokeh.embed import components
from bokeh.resources import Resources
from bokeh.io import output_notebook
output_notebook()

print('Notebook last executed on', datetime.date.today(), 'using iprPy version', iprPy.__version__)

Bokeh version = 1.4.0


Loading BokehJS ...

Notebook last executed on 2019-12-03 using iprPy version 0.9.1


#### 1.2. Default calculation setup

In [2]:
# Specify calculation style
calc_style = 'diatom_scan'

# If workingdir is already set, then do nothing (already in correct folder)
try:
    workingdir = workingdir

# Change to workingdir if not already there
except:
    workingdir = Path('calculationfiles', calc_style)
    if not workingdir.is_dir():
        workingdir.mkdir(parents=True)
    os.chdir(workingdir)

### 2. Calculation setup/definition

#### 2.1 Input widget functions

These simply define the widgets for inputting the calculation parameters

- __widget_diatom_scan__ allows all parameters for diatom_scan to be specified

In [3]:
def widget_diatom_scan(data):
    
    # Define library database
    database = iprPy.load_database(style='library')
    
    # Fetch potentials
    potentials, potentials_df = database.get_records(style='potential_LAMMPS', return_df=True)
    potentials = np.asarray(potentials)
    
    # Set initial potential
    pot_record = potentials[0]
    pot_dir = Path(database.host, 'potential_LAMMPS', pot_record.name)
    potential = lmp.Potential(pot_record.content, pot_dir)
    
    # Set default data values
    data['lammps_command'] = 'lmp_serial'
    data['potential'] = potential
    data['symbols'] = [potential.symbols[0], potential.symbols[0]]
    data['rmin'] = 0.02
    data['rmax'] = 6.0
    data['rsteps'] = 300
    
    # Define widgets and outputs
    widget_lammps_command = Text(value=data['lammps_command'], description='LAMMPS:')
    widget_potential = Dropdown(options=potentials_df.id.tolist(), description='Potential:')
    widget_symbol1 = Dropdown(options=potential.symbols, value=potential.symbols[0], description='Symbol 1:')
    widget_symbol2 = Dropdown(options=potential.symbols, value=potential.symbols[0], description='Symbol 2:')
    widget_rmin = BoundedFloatText(value=data['rmin'], min=0, max=data['rmax'], description='Min r:')
    widget_rmax = BoundedFloatText(value=data['rmax'], min=data['rmin'], max=10, description='Max r:')
    widget_rsteps = BoundedIntText(value=data['rsteps'], min=2, max=1000, description='Num r steps:')
    data_out = Output()
    
    def showdata():
        with data_out:
            clear_output()
            print(data)
    
    def lammps_command_event(*args):
        """Event manager"""
        data['lammps_command'] = widget_lammps_command.value
        showdata()
    widget_lammps_command.observe(lammps_command_event)
    
    def potentials_action(*args):
        """Event manager"""
        pot_record = potentials[potentials_df.id == widget_potential.value][0]
        pot_dir = Path(database.host, 'potential_LAMMPS', pot_record.name)
        potential = lmp.Potential(pot_record.content, pot_dir)
        data['potential'] = potential
        widget_symbol1.options = potential.symbols
        widget_symbol2.options = potential.symbols
        showdata()
    widget_potential.observe(potentials_action)
    
    def symbols_action(*args):
        """Event manager"""
        data['symbols'] = [widget_symbol1.value, widget_symbol2.value]
        showdata()
    widget_symbol1.observe(symbols_action)
    widget_symbol2.observe(symbols_action)
    
    def rmin_action(*args):
        """Event manager"""
        data['rmin'] = widget_rmin.value
        widget_rmax.min = widget_rmin.value
        showdata()
    widget_rmin.observe(rmin_action)
    
    def rmax_action(*args):
        """Event manager"""
        data['rmax'] = widget_rmax.value
        widget_rmin.max = widget_rmax.value
        showdata()
    widget_rmax.observe(rmax_action)
        
    def rsteps_event(*args):
        """Event manager"""
        data['rsteps'] = widget_rsteps.value
        showdata()
    widget_rsteps.observe(rsteps_event)    
    
    showdata()
    
    display(
        widget_lammps_command,
        widget_potential,
        widget_symbol1,
        widget_symbol2,
        widget_rmin,
        widget_rmax,
        widget_rsteps,
        data_out
    )

#### 2.2. LAMMPS input templates

These are template LAMMPS input scripts used by the calculation functions

- __run0.template__ computes the energy of a system using a potential without any relaxations

In [4]:
with open('run0.template', 'w') as f:
    f.write("""#LAMMPS input script that evaluates a system's energy without relaxing

<atomman_system_pair_info>

thermo_style custom step pe
thermo_modify format float %.13e

run 0""")

#### 2.3. Calculation functions

These are the calculation functions 

- __diatom__ computes the interaction energy between two atoms over a range of interatomic spaces

In [5]:
def diatom(lammps_command, potential, symbols,
           mpi_command=None, 
           rmin=uc.set_in_units(0.02, 'angstrom'), 
           rmax=uc.set_in_units(6.0, 'angstrom'), rsteps=300):
    """
    Performs a diatom energy scan over a range of interatomic spaces, r.
    
    Parameters
    ----------
    lammps_command :str
        Command for running LAMMPS.
    potential : atomman.lammps.Potential
        The LAMMPS implemented potential to use.
    symbols : list
        The potential symbols associated with the two atoms in the diatom.
    mpi_command : str, optional
        The MPI command for running LAMMPS in parallel.  If not given, LAMMPS
        will run serially.
    rmin : float, optional
        The minimum r spacing to use (default value is 0.02 angstroms).
    rmax : float, optional
        The maximum r spacing to use (default value is 6.0 angstroms).
    rsteps : int, optional
        The number of r spacing steps to evaluate (default value is 300).
    
    Returns
    -------
    dict
        Dictionary of results consisting of keys:
        
        - **'r_values'** (*numpy.array of float*) - All interatomic spacings,
          r, explored.
        - **'energy_values'** (*numpy.array of float*) - The computed potential
          energies for each r value.
    """
    try:
        # Get script's location if __file__ exists
        script_dir = Path(__file__).parent
    except:
        # Use cwd otherwise
        script_dir = Path.cwd()
 
    # Build lists of values
    r_values = np.linspace(rmin, rmax, rsteps)
    energy_values = np.empty(rsteps)
    
    # Define atype based on symbols
    symbols = iprPy.tools.aslist(symbols)
    if len(symbols) == 1:
        atype = [1, 1]
    elif len(symbols) == 2:
        atype = [1, 2]
    else:
        raise ValueError('symbols must have one or two values')
    
    # Initialize system (will shift second atom's position later...)
    box = am.Box.cubic(a = rmax + 1)
    atoms = am.Atoms(atype=atype, pos=[[0.1, 0.1, 0.1], [0.1, 0.1, 0.1]])
    system = am.System(atoms=atoms, box=box, pbc=[False, False, False], symbols=symbols)

    # Add charges if required
    if potential.atom_style == 'charge':
        system.atoms.prop_atype('charge', potential.charges(system.symbols))

    # Get lammps units
    lammps_units = lmp.style.unit(potential.units)

    # Define lammps variables
    lammps_variables = {}

    # Loop over values
    for i in range(rsteps):
        
        # Shift second atom's x position
        system.atoms.pos[1] = np.array([0.1 + r_values[i], 0.1, 0.1])

        # Save configuration
        system_info = system.dump('atom_data', f='diatom.dat',
                                  potential=potential,
                                  return_pair_info=True)
        lammps_variables['atomman_system_pair_info'] = system_info
        
        # Write lammps input script
        template_file = Path(script_dir, 'run0.template')
        lammps_script = 'run0.in'
        with open(template_file) as f:
            template = f.read()
        with open(lammps_script, 'w') as f:
            f.write(iprPy.tools.filltemplate(template, lammps_variables,
                                             '<', '>'))
        
        # Run lammps and extract data
        try:
            output = lmp.run(lammps_command, lammps_script, mpi_command)
        except:
            energy_values[i] = np.nan
        else:
            energy = output.simulations[0]['thermo'].PotEng.values[-1]
            energy_values[i] = uc.set_in_units(energy, lammps_units['energy'])

    if len(energy_values[np.isfinite(energy_values)]) == 0:
        raise ValueError('All LAMMPS runs failed. Potential likely invalid or incompatible.')
    
    # Collect results
    results_dict = {}
    results_dict['r_values'] = r_values
    results_dict['energy_values'] = energy_values
    
    return results_dict

#### 2.4 Display functions

These are pre-defined functions for displaying results

- __show_E_vs_r_plot__ plots the energies versus interatomic spacing in the "normal" regime

In [14]:
def show_E_vs_r_plot(input_dict, results_dict):
    energy = results_dict['energy_values']
    r = results_dict['r_values']

    Emin = floor(energy.min())
    if Emin < -10: 
        Emin = -10

    plot = figure(title = f'Diatom energy scan for {input_dict["potential"]}',
                  plot_width = 800,
                  plot_height = 600,
                  x_range = [input_dict['rmin'], input_dict['rmax']],
                  y_range = [Emin, 0],              
                  x_axis_label=f'r (Angstroms)', 
                  y_axis_label=f'Interaction Energy (eV)')

    plot.line(r, energy, line_width=2, legend_label='-'.join(input_dict['symbols']))  
    plot.legend.location = "bottom_right"    

    show(plot)

### 3. Calculation Input Parameters

- __LAMMPS__: LAMMPS executable to use
- __Potential__: Name of the interatomic potential to use
- __Symbol 1 and 2__: The elemental models from the potential to use for the two atoms
- __Min and Max r__: The range of interatomic distances to explore
- __Num r steps__: The number of measurements to make

In [15]:
input_dict = {}
widget_diatom_scan(input_dict)

Text(value='lmp_serial', description='LAMMPS:')

Dropdown(description='Potential:', options=('1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1', '1985--Stillinger-F-H--Si…

Dropdown(description='Symbol 1:', options=('Cu', 'Ni'), value='Cu')

Dropdown(description='Symbol 2:', options=('Cu', 'Ni'), value='Cu')

BoundedFloatText(value=0.02, description='Min r:', max=6.0)

BoundedFloatText(value=6.0, description='Max r:', max=10.0, min=0.02)

BoundedIntText(value=300, description='Num r steps:', max=1000, min=2)

Output()

### 4. Run calculation function(s) and display results

In [16]:
results_dict = diatom(input_dict['lammps_command'],
                      input_dict['potential'],
                      input_dict['symbols'],
                      rmin = input_dict['rmin'], 
                      rmax = input_dict['rmax'], 
                      rsteps = input_dict['rsteps'])

show_E_vs_r_plot(input_dict, results_dict)